In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:2]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/intel-image-classification/seg_train/seg_train/mountain/14986.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/mountain/3138.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/street/1269.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/street/6241.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/buildings/2193.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/buildings/11378.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/sea/19812.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/sea/16916.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/forest/7981.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/forest/3863.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/glacier/12666.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/glacier/13288.jpg
/kaggle/input/intel-image-classification/seg_pred/seg_pred/6234.

In [2]:

#!pip uninstall -y protobuf
#!pip install protobuf==3.20.2

In [3]:
#!pip install -q dask==2021.11.2
#!pip install -q lightning-bolts["extra"]
#!pip install git+https://github.com/PytorchLightning/lightning-bolts.git@master --upgrade
!pip install -q lightning-bolts==0.6.0.post1
!pip install -q lightning-lite==1.8.0
!pip install -q lightning-utilities==0.3.0
!pip install -q pytorch-lightning<1.9.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 1.9.4 requires lightning-utilities>=0.6.0.post0, but you have lightning-utilities 0.3.0 which is incompatible.
/bin/bash: 1.9.0: No such file or directory


In [4]:
#from pytorch_lightning.loggers import TensorBoardLogger
#from pytorch_lightning.callbacks import EarlyStopping
#from pytorch_lightning.loggers.base import LightningLoggerBase

In [5]:
import torch
import pytorch_lightning as pl
from pl_bolts.models.autoencoders import VAE
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities import rank_zero_only

ImportError: cannot import name 'ModuleAvailableCache' from 'lightning_utilities.core.imports' (/opt/conda/lib/python3.7/site-packages/lightning_utilities/core/imports.py)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.image_list[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        # Get class label from image directory name
        label = os.path.basename(self.root_dir)

        return image

In [ ]:
class VAEModel(VAE):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.save_hyperparameters()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams['lr'])
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.hparams['max_epochs'])
        return {"optimizer": optimizer, "lr_scheduler": scheduler}

    def training_step(self, batch, batch_idx):
        loss = super().training_step((batch, 0), batch_idx)
        #self.log('train_loss', loss, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss = super().validation_step((batch, 0), batch_idx)
        #self.log('val_loss', loss, on_epoch=True)
    
    def test_step(self, batch, batch_idx):
        loss = super().test_step((batch, 0), batch_idx)
        #self.log('test_loss', loss, on_epoch=True)

    def train_dataloader(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((self.input_height, self.input_height)),])
        train_dataset = CustomDataset(root_dir='/kaggle/input/intel-image-classification/seg_train/seg_train/mountain/', transform=transform)
        return DataLoader(train_dataset, batch_size=self.hparams['batch_size'])

    def val_dataloader(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((self.input_height, self.input_height)),])
        val_dataset = CustomDataset(root_dir='/kaggle/input/intel-image-classification/seg_test/seg_test/mountain/', transform=transform)
        return DataLoader(val_dataset, batch_size=self.hparams['batch_size'])

    def test_dataloader(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((self.input_height, self.input_height)),])
        test_dataset = CustomDataset(root_dir='/kaggle/input/intel-image-classification/seg_test/seg_test/mountain/', transform=transform)
        return DataLoader(test_dataset, batch_size=self.hparams['batch_size'])

    def on_train_start(self):
        if self.hparams['pretrained_model_path']:
            self.load_state_dict(torch.load(self.hparams['pretrained_model_path']))

    def generate_images(self, num_images):
        z = torch.randn(num_images, self.latent_dim, device=self.device)
        with torch.no_grad():
            generated_images = self.decoder(z)
        generated_images = generated_images.permute(0, 2, 3, 1).cpu().numpy()
        generated_images = (generated_images * 255).astype(np.uint8)
        return generated_images

In [ ]:
if __name__ == '__main__':
    hparams = {
        'input_height': 64,
        'latent_dim': 768,
        'max_epochs': 250,
        'batch_size': 128,
        'pretrained_model_path': None
    }
    checkpoint_callback = ModelCheckpoint(
        dirpath='./models',
        filename='vae-{epoch:02d}-{step:02d}-{val_loss:.2f}',
        save_top_k=3,
        verbose=True,
        monitor='val_loss',
        mode='min',
        every_n_epochs=25  # set n to the desired number of iterations
    )
    #logger = TensorBoardLogger(save_dir="logs/")
    # Initialize a trainer instance and train the model
    trainer = pl.Trainer(
        max_epochs=250,
        accelerator='gpu',
        devices=1,
        precision=16,
        #limit_train_batches=0.1,
        #limit_val_batches=0.1,
        auto_lr_find=True,
        callbacks=[checkpoint_callback],
        sync_batchnorm=True,
    )
    model = VAEModel(**hparams)
    trainer.fit(model)

In [ ]:
import matplotlib.pyplot as plt

# Generate some images
generated_images = model.generate_images(num_images=4)

print(generated_images[0].shape)

# Plot the generated images
fig, axs = plt.subplots(1, 4)
for i in range(4):
    axs[i].imshow(generated_images[i])
    axs[i].axis('off')
plt.show()


In [ ]:
print(hparams['latent_dim'])